# Overview

This notebook combines two models:

Score 0.467: [https://www.kaggle.com/andretugan/pre-trained-roberta-solution-in-pytorch](https://www.kaggle.com/andretugan/pre-trained-roberta-solution-in-pytorch)

Score 0.468: [https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-infer-3](https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-infer-3)

# Model 1

In [1]:
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig

from sklearn.model_selection import KFold
from sklearn.svm import SVR
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler, Sampler
import gc
gc.enable()

In [2]:
BATCH_SIZE = 32
MAX_LEN = 250
EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
ROBERTA_PATH = "/kaggle/input/roberta-base"
TOKENIZER_PATH = "/kaggle/input/roberta-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
test_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

In [5]:
from IPython.core.magic import register_cell_magic
import os
from pathlib import Path

## define custom magic to save most useful classes and use them in inference notebook 
## instead of copying the code every time you have changes in the classes
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)
    
Path('/kaggle/working/scripts').mkdir(exist_ok=True)
models_dir = Path('/kaggle/working/models')
models_dir.mkdir(exist_ok=True)

In [6]:
%%write_and_run scripts/config.py

class Config:
    model_name = 'roberta-large'
    output_hidden_states = True
    epochs = 5
#     evaluate_interval = 40
    batch_size = 8
    device = 'cuda'
    seed = 42
    max_len = 256
    lr = 1e-5
    wd = 0.01
#     eval_schedule = [(float('inf'), 40), (0.5, 30), (0.49, 20), (0.48, 10), (0.47, 3), (0, 0)]
    eval_schedule = [(float('inf'), 40), (0.47, 20), (0.46, 10), (0, 0)]

    gradient_accumulation = 2

# Dataset

In [7]:
class LitDataset(Dataset):
    def __init__(self, df, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only
        self.text = df.excerpt.tolist()
        #self.text = [text.replace("\n", " ") for text in self.text]
        
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)        
    
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, target)

# Model
The model is inspired by the one from [Maunish](https://www.kaggle.com/maunish/clrp-roberta-svm).

In [8]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(768, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(768, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [9]:
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

# Inference

In [10]:
test_dataset = LitDataset(test_df, inference_only=True)

In [11]:
NUM_MODELS = 5

all_predictions = np.zeros((NUM_MODELS, len(test_df)))



test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for model_index in range(NUM_MODELS):            
    model_path = f"../input/commonlit-roberta-0467/model_{model_index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModel()
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))    
    model.to(DEVICE)
        
    all_predictions[model_index] = predict(model, test_loader)
            
    del model
    gc.collect()


Using ../input/commonlit-roberta-0467/model_1.pth

Using ../input/commonlit-roberta-0467/model_2.pth

Using ../input/commonlit-roberta-0467/model_3.pth

Using ../input/commonlit-roberta-0467/model_4.pth

Using ../input/commonlit-roberta-0467/model_5.pth


In [12]:
model1_predictions = all_predictions.mean(axis=0)

In [13]:
ROBERTA_LARGE_PATH = "../input/clrp-roberta-large/clrp_roberta_large"

In [14]:
class LitModelBig(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_LARGE_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_LARGE_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(1024, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

# Model 3

In [15]:
NUM_MODELS = 5

all_predictions = np.zeros((NUM_MODELS, len(test_df)))



test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for model_index in range(NUM_MODELS):            
    model_path = f"../input/roberta-large-250-seeds-2e55e57e5-0-261-309/model_{model_index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModelBig()
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))    
    model.to(DEVICE)
        
    all_predictions[model_index] = predict(model, test_loader)
            
    del model
    gc.collect()


Using ../input/roberta-large-250-seeds-2e55e57e5-0-261-309/model_1.pth


Some weights of RobertaModel were not initialized from the model checkpoint at ../input/clrp-roberta-large/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using ../input/roberta-large-250-seeds-2e55e57e5-0-261-309/model_2.pth


Some weights of RobertaModel were not initialized from the model checkpoint at ../input/clrp-roberta-large/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using ../input/roberta-large-250-seeds-2e55e57e5-0-261-309/model_3.pth


Some weights of RobertaModel were not initialized from the model checkpoint at ../input/clrp-roberta-large/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using ../input/roberta-large-250-seeds-2e55e57e5-0-261-309/model_4.pth


Some weights of RobertaModel were not initialized from the model checkpoint at ../input/clrp-roberta-large/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using ../input/roberta-large-250-seeds-2e55e57e5-0-261-309/model_5.pth


Some weights of RobertaModel were not initialized from the model checkpoint at ../input/clrp-roberta-large/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model3_predictions = all_predictions.mean(axis=0)

# FineTune Roberta Large

In [17]:
%%write_and_run scripts/dataset.py

from torch.utils.data import Dataset
import torch

def convert_examples_to_features(text, tokenizer, max_len):

    tok = tokenizer.encode_plus(
        text, 
        max_length=max_len, 
        truncation=True,
        padding='max_length',
    )
    return tok


class CLRPDataset(Dataset):
    def __init__(self, data, tokenizer, max_len, is_test=False):
        self.data = data
        self.excerpts = self.data.excerpt.tolist()
        if not is_test:
            self.targets = self.data.target.tolist()
            
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        if not self.is_test:
            excerpt = self.excerpts[item]
            label = self.targets[item]
            features = convert_examples_to_features(
                excerpt, self.tokenizer, self.max_len
            )
            return {
                'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
                'label':torch.tensor(label, dtype=torch.float),
            }
        else:
            excerpt = self.excerpts[item]
            features = convert_examples_to_features(
                excerpt, self.tokenizer, self.max_len
            )
            return {
                'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
            }

In [18]:
def make_dataloader(data, tokenizer, is_train=True):
    dataset = CLRPDataset(data, tokenizer=tokenizer, max_len=Config.max_len,is_test=True)
    if is_train:
        sampler = RandomSampler(dataset)
    else:
        sampler = SequentialSampler(dataset)

    batch_dataloader = DataLoader(dataset, sampler=sampler, batch_size=Config.batch_size, pin_memory=True, collate_fn=DynamicPadCollate())
    return batch_dataloader

In [19]:
class DynamicPadCollate:
    def __call__(self,batch):
                
        out = {'input_ids' :[],
               'attention_mask':[],
                'label':[]
        }
        
        for i in batch:
            for k,v in i.items():
                out[k].append(v)
                
        max_pad =0

        for p in out['input_ids']:
            if max_pad < len(p):
                max_pad = len(p)
                    

        for i in range(len(batch)):
            
            input_id = out['input_ids'][i]
            att_mask = out['attention_mask'][i]
            text_len = len(input_id)
            
            out['input_ids'][i] = (out['input_ids'][i].tolist() + [1] * (max_pad - text_len))[:max_pad]
            out['attention_mask'][i] = (out['attention_mask'][i].tolist() + [0] * (max_pad - text_len))[:max_pad]
        
        out['input_ids'] = torch.tensor(out['input_ids'],dtype=torch.long)
        out['attention_mask'] = torch.tensor(out['attention_mask'],dtype=torch.long)
        out['label'] = torch.tensor(out['label'],dtype=torch.float)
        
        return out

In [20]:
%%write_and_run scripts/model.py

import torch
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, h_size, hidden_dim=512):
        super().__init__()
        self.W = nn.Linear(h_size, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        
    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

class CLRPModel(nn.Module):
    def __init__(self,transformer,config):
        super(CLRPModel,self).__init__()
        self.h_size = config.hidden_size
        self.transformer = transformer
        self.head = AttentionHead(self.h_size*4)
        self.linear = nn.Linear(self.h_size*2, 1)
        self.linear_out = nn.Linear(self.h_size*8, 1)

              
    def forward(self, input_ids, attention_mask):
        transformer_out = self.transformer(input_ids, attention_mask)
       
        all_hidden_states = torch.stack(transformer_out.hidden_states)
        cat_over_last_layers = torch.cat(
            (all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]),-1
        )
        
        cls_pooling = cat_over_last_layers[:, 0]   
        head_logits = self.head(cat_over_last_layers)
        y_hat = self.linear_out(torch.cat([head_logits, cls_pooling], -1))
        
        return y_hat
def create_optimizer(model):
    named_parameters = list(model.named_parameters())    
    
    roberta_parameters = named_parameters[:389]    
    attention_parameters = named_parameters[391:395]
    regressor_parameters = named_parameters[395:]
        
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]

    parameters = []
    parameters.append({"params": attention_group})
    parameters.append({"params": regressor_group})
    
    # increase lr every second layer
    increase_lr_every_k_layer = 1
    lrs = np.linspace(1, 5, 24 // increase_lr_every_k_layer)
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if "bias" in name else 0.01
        splitted_name = name.split('.')
        lr = Config.lr
        if len(splitted_name) >= 4 and str.isdigit(splitted_name[3]):
            layer_num = int(splitted_name[3])
            lr = lrs[layer_num // increase_lr_every_k_layer] * Config.lr 

        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})

    return optim.AdamW(parameters)

In [21]:
def predict_finetuned(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, batch in enumerate(data_loader):
            input_ids, attention_mask = batch['input_ids'], batch['attention_mask']
            #(input_ids, attention_mask) 
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

In [22]:
NUM_MODELS = 5
import copy
all_predictions = np.zeros((NUM_MODELS, len(test_df)))

tokenizer = AutoTokenizer.from_pretrained(ROBERTA_LARGE_PATH)
config = AutoConfig.from_pretrained(ROBERTA_LARGE_PATH)
config.update({
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7,
            "output_hidden_states": True
            }) 
test_dl = make_dataloader(test_df, tokenizer, is_train=False)
transformer = AutoModel.from_pretrained(ROBERTA_LARGE_PATH, config=config)  

for model_index in range(NUM_MODELS):            
    model_path = f"../input/finetune-roberta-large-public/models/best_model_{model_index}.pth" #../input/finetune-roberta-large-public/models/best_model_0.pth
    print(f"\nUsing {model_path}")
                        
    model = CLRPModel(transformer, config)
    #model.load_state_dict(torch.load(model_path, map_location=DEVICE))    
    model.load_state_dict(torch.load(model_path,DEVICE))
    model.to(DEVICE)
        
    all_predictions[model_index] = predict_finetuned(model, test_dl)
            
    del model
    gc.collect()

Some weights of RobertaModel were not initialized from the model checkpoint at ../input/clrp-roberta-large/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using ../input/finetune-roberta-large-public/models/best_model_0.pth

Using ../input/finetune-roberta-large-public/models/best_model_1.pth

Using ../input/finetune-roberta-large-public/models/best_model_2.pth

Using ../input/finetune-roberta-large-public/models/best_model_3.pth

Using ../input/finetune-roberta-large-public/models/best_model_4.pth


In [23]:
model5_predictions = all_predictions.mean(axis=0)

# Model 2
Imported from [https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-infer-3](https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-infer-3)

In [24]:
test = test_df

from glob import glob
import os
import matplotlib.pyplot as plt
import json
from collections import defaultdict


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.optimizer import Optimizer
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import (
    Dataset, DataLoader, 
    SequentialSampler, RandomSampler
)
from transformers import RobertaConfig
from transformers import (
    get_cosine_schedule_with_warmup, 
    get_cosine_with_hard_restarts_schedule_with_warmup
)
from transformers import RobertaTokenizer
from transformers import RobertaModel
from IPython.display import clear_output
from tqdm import tqdm, trange

def convert_examples_to_features(data, tokenizer, max_len, is_test=False):
    data = data.replace('\n', '')
    tok = tokenizer.encode_plus(
        data, 
        max_length=max_len, 
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    curr_sent = {}
    padding_length = max_len - len(tok['input_ids'])
    curr_sent['input_ids'] = tok['input_ids'] + ([0] * padding_length)
    curr_sent['token_type_ids'] = tok['token_type_ids'] + \
        ([0] * padding_length)
    curr_sent['attention_mask'] = tok['attention_mask'] + \
        ([0] * padding_length)
    return curr_sent

class DatasetRetriever(Dataset):
    def __init__(self, data, tokenizer, max_len, is_test=False):
        self.data = data
        self.excerpts = self.data.excerpt.values.tolist()
        self.tokenizer = tokenizer
        self.is_test = is_test
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        if not self.is_test:
            excerpt, label = self.excerpts[item], self.targets[item]
            features = convert_examples_to_features(
                excerpt, self.tokenizer, 
                self.max_len, self.is_test
            )
            return {
                'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
                'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
                'label':torch.tensor(label, dtype=torch.double),
            }
        else:
            excerpt = self.excerpts[item]
            features = convert_examples_to_features(
                excerpt, self.tokenizer, 
                self.max_len, self.is_test
            )
            return {
                'input_ids':torch.tensor(features['input_ids'], dtype=torch.long),
                'token_type_ids':torch.tensor(features['token_type_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(features['attention_mask'], dtype=torch.long),
            }

class CommonLitModel(nn.Module):
    def __init__(
        self, 
        model_name, 
        config,  
        multisample_dropout=False,
        output_hidden_states=False
    ):
        super(CommonLitModel, self).__init__()
        self.config = config
        self.roberta = RobertaModel.from_pretrained(
            model_name, 
            output_hidden_states=output_hidden_states
        )
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        if multisample_dropout:
            self.dropouts = nn.ModuleList([
                nn.Dropout(0.5) for _ in range(5)
            ])
        else:
            self.dropouts = nn.ModuleList([nn.Dropout(0.3)])
        self.regressor = nn.Linear(config.hidden_size, 1)
        self._init_weights(self.layer_norm)
        self._init_weights(self.regressor)
 
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
 
    def forward(
        self, 
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        sequence_output = outputs[1]
        sequence_output = self.layer_norm(sequence_output)
 
        # multi-sample dropout
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                logits = self.regressor(dropout(sequence_output))
            else:
                logits += self.regressor(dropout(sequence_output))
        
        logits /= len(self.dropouts)
 
        # calculate loss
        loss = None
        if labels is not None:
            loss_fn = torch.nn.MSELoss()
            logits = logits.view(-1).to(labels.dtype)
            loss = torch.sqrt(loss_fn(logits, labels.view(-1)))
        
        output = (logits,) + outputs[1:]
        return ((loss,) + output) if loss is not None else output

def make_model(model_name, num_labels=1):
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    config = RobertaConfig.from_pretrained(model_name)
    config.update({'num_labels':num_labels})
    model = CommonLitModel(model_name, config=config)
    return model, tokenizer

def make_loader(
    data, 
    tokenizer, 
    max_len,
    batch_size,
):
    
    test_dataset = DatasetRetriever(data, tokenizer, max_len, is_test=True)
    test_sampler = SequentialSampler(test_dataset)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size // 2, 
        sampler=test_sampler, 
        pin_memory=False, 
        drop_last=False, 
        num_workers=0
    )

    return test_loader

class Evaluator:
    def __init__(self, model, scalar=None):
        self.model = model
        self.scalar = scalar

    def evaluate(self, data_loader, tokenizer):
        preds = []
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch_idx, batch_data in enumerate(data_loader):
                input_ids, attention_mask, token_type_ids = batch_data['input_ids'], \
                    batch_data['attention_mask'], batch_data['token_type_ids']
                input_ids, attention_mask, token_type_ids = input_ids.cuda(), \
                    attention_mask.cuda(), token_type_ids.cuda()
                
                if self.scalar is not None:
                    with torch.cuda.amp.autocast():
                        outputs = self.model(
                            input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids
                        )
                else:
                    outputs = self.model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids
                    )
                
                logits = outputs[0].detach().cpu().numpy().squeeze().tolist()
                preds += logits
        return preds

def config(fold, model_name, load_model_path):
    torch.manual_seed(2021)
    torch.cuda.manual_seed(2021)
    torch.cuda.manual_seed_all(2021)
    
    max_len = 250
    batch_size = 8

    model, tokenizer = make_model(
        model_name=model_name, 
        num_labels=1
    )
    model.load_state_dict(
        torch.load(f'{load_model_path}/model{fold}.bin')
    )
    test_loader = make_loader(
        test, tokenizer, max_len=max_len,
        batch_size=batch_size
    )

    if torch.cuda.device_count() >= 1:
        print('Model pushed to {} GPU(s), type {}.'.format(
            torch.cuda.device_count(), 
            torch.cuda.get_device_name(0))
        )
        model = model.cuda() 
    else:
        raise ValueError('CPU training is not supported')

    # scaler = torch.cuda.amp.GradScaler()
    scaler = None
    return (
        model, tokenizer, 
        test_loader, scaler
    )

def run(fold=0, model_name=None, load_model_path=None):
    model, tokenizer, \
        test_loader, scaler = config(fold, model_name, load_model_path)
    
    import time

    evaluator = Evaluator(model, scaler)

    test_time_list = []

    torch.cuda.synchronize()
    tic1 = time.time()

    preds = evaluator.evaluate(test_loader, tokenizer)

    torch.cuda.synchronize()
    tic2 = time.time() 
    test_time_list.append(tic2 - tic1)
    
    del model, tokenizer, test_loader, scaler
    gc.collect()
    torch.cuda.empty_cache()
    
    return preds

#pred_df1 = pd.DataFrame()
#pred_df2 = pd.DataFrame()
#pred_df3 = pd.DataFrame()
#for fold in tqdm(range(5)):
    #pred_df1[f'fold{fold}'] = run(fold, '../input/roberta-base/', '../input/commonlit-roberta-base-i/')
    #pred_df2[f'fold{fold+5}'] = run(fold, '../input/robertalarge/', '../input/roberta-large-itptfit/')
    #pred_df3[f'fold{fold+10}'] = run(fold, '../input/robertalarge/', '../input/commonlit-roberta-large-ii/')

In [25]:
#pred_df1 = np.array(pred_df1)
#pred_df2 = np.array(pred_df2)
#pred_df3 = np.array(pred_df3)
#model2_predictions = pred_df2.mean(axis=1)# + (pred_df1.mean(axis=1)*5) + (pred_df3.mean(axis=1) * 9)) / 16

# Model 4

In [26]:
from numba import cuda 
import pandas as pd

import re
import os
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
from transformers import RobertaTokenizer, TFRobertaModel
from kaggle_datasets import KaggleDatasets
tf.get_logger().setLevel(logging.ERROR)
from kaggle_datasets import KaggleDatasets
from transformers import AutoTokenizer

from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression, Ridge

import tensorflow as tf 
from tensorflow.keras.layers import Input,LSTM,Bidirectional,Embedding,Dense, Conv1D, Dropout , MaxPool1D , MaxPooling1D, GlobalAveragePooling2D , GlobalAveragePooling1D , GlobalMaxPooling1D , concatenate , Flatten
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Model,load_model,save_model , model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping ,LearningRateScheduler
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from transformers import TFBertModel, BertTokenizerFast , BertTokenizer , RobertaTokenizerFast , TFRobertaModel , RobertaConfig , TFAutoModel , AutoTokenizer

In [27]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu,True)
        
max_len = 250
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

MODEL=['bert-base-uncased' , 'roberta-base']

model_name = MODEL[1]

path=[
    "../input/commonlitreadabilityprize/sample_submission.csv",
    "../input/commonlitreadabilityprize/test.csv",
    "../input/commonlitreadabilityprize/train.csv"
]

df_train = pd.read_csv(path[2])
df_test = pd.read_csv(path[1])
df_ss = pd.read_csv(path[0])
                         
df_train = df_train.drop(['url_legal','license','standard_error'],axis='columns')
df_test = df_test.drop(['url_legal','license'],axis='columns')
X= df_train['excerpt']
y=df_train['target'].values

X_test = df_test['excerpt']

tokenizer1 = AutoTokenizer.from_pretrained("../input/huggingface-roberta-variants/roberta-base/roberta-base")

print('tokenization')
train_embeddings = tokenizer1(X.to_list(), truncation = True , padding = 'max_length' , max_length=max_len)
test_embeddings = tokenizer1(X_test.to_list() , truncation = True , padding = 'max_length' , max_length = max_len)
                         
@tf.function
def map_function(encodings):
    input_ids = encodings['input_ids']
    
    return {'input_word_ids': input_ids}

print("generating train and test")    
train = tf.data.Dataset.from_tensor_slices((train_embeddings))
train = (
            train
            .map(map_function, num_parallel_calls=AUTOTUNE)
            .batch(16)
            .prefetch(AUTOTUNE)
        )


test = tf.data.Dataset.from_tensor_slices((test_embeddings))
test = (
        test
        .map(map_function, num_parallel_calls = AUTOTUNE)
        .batch(16)
        .prefetch(AUTOTUNE)
    )
                         
                         
def build_roberta_base_model(max_len=max_len ):
    
    transformer = TFAutoModel.from_pretrained("../input/huggingface-roberta-variants/roberta-base/roberta-base")
    
    input_word_ids = tf.keras.layers.Input(shape = (max_len, ), dtype = tf.int32, name = 'input_word_ids')
    sequence_output = transformer(input_word_ids)[0]
    
    # We only need the cls_token, resulting in a 2d array
    cls_token = sequence_output[:, 0, :]
    output = tf.keras.layers.Dense(1, activation = 'linear', dtype = 'float32')(cls_token)
    
    model = tf.keras.models.Model(inputs = [input_word_ids], outputs = output)
    
    return model
                         
ragnar_model = build_roberta_base_model()
def feature_extractor(path):
    print("loading weights")
    ragnar_model.load_weights(path)
    x= ragnar_model.layers[-3].output
    model = Model(inputs = ragnar_model.inputs , outputs = x)
    return model
                         
def get_preds(model,train,test):
    print("Extracting Features from train data")
    train_features = model.predict( train , verbose =1)
    train_features = train_features.last_hidden_state
    train_features = train_features[: , 0 , :]
    print("Extracting Features from train data")
    test_features = model.predict( test , verbose =1)
    test_features = test_features.last_hidden_state
    test_features = test_features[: , 0 , :]
    
    return np.array(train_features , dtype= np.float16) , np.array(test_features , dtype= np.float16) 
                         
#model weight paths
paths=["../input/commonlit-readability-roberta-base/Roberta_Base_123_1.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_2.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_3.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_4.h5",
       "../input/commonlit-readability-roberta-base/Roberta_Base_123_5.h5"
      ]
                         
#1
extraction_model = feature_extractor(paths[0])
train_embeddings1 , test_embeddings1 = get_preds(extraction_model , train , test)
                         
#2
extraction_model = feature_extractor(paths[1])
train_embeddings2 , test_embeddings2 = get_preds(extraction_model , train , test)
                         
#3
extraction_model = feature_extractor(paths[2])
train_embeddings3 , test_embeddings3 = get_preds(extraction_model , train , test)
                         
#4
extraction_model = feature_extractor(paths[3])
train_embeddings4 , test_embeddings4 = get_preds(extraction_model , train , test)
                         
#5
extraction_model = feature_extractor(paths[4])
train_embeddings5 , test_embeddings5 = get_preds(extraction_model , train , test)
                         
def get_preds(train_embeddings , test_embeddings):
    scores=[]
    kfold = KFold(n_splits=5, shuffle= True , random_state=2021)
    iteration=1
    preds = np.zeros((test_embeddings.shape[0]))
    for train_idx, test_idx in kfold.split(train_embeddings,y):
        print(f'running iteration {iteration}')
        X_train = train_embeddings[train_idx]
        X_test = train_embeddings[test_idx]
        y_train = y[train_idx]
        y_test = y[test_idx]

        regression_model = Ridge()
        
        regression_model.fit(X_train,y_train)
        y_pred = regression_model.predict(X_test)

        score = np.sqrt(mse(y_pred,y_test))
        scores.append(score)
        print(f'Fold {iteration} , rmse score: {score}')
        y_preds = regression_model.predict(test_embeddings)
        y_preds=y_preds.reshape(-1)
        preds+=y_preds  
        iteration += 1

    print(f"the average rmse is {np.mean(scores)}")
    return np.array(preds)/5  
                         
                         
print("***********predicting***********")
preds1 = get_preds(train_embeddings1,test_embeddings1)
print("***********predicting***********")
preds2 = get_preds(train_embeddings2,test_embeddings2)
print("***********predicting***********")
preds3 = get_preds(train_embeddings3,test_embeddings3)
print("***********predicting***********")
preds4 = get_preds(train_embeddings4,test_embeddings4)
print("***********predicting***********")
preds5 = get_preds(train_embeddings5,test_embeddings5)

preds_ragnar=(preds1+preds2+preds3+preds4+preds5)/5
#preds_ragnar = preds.tolist()

tokenization
generating train and test


Some layers from the model checkpoint at ../input/huggingface-roberta-variants/roberta-base/roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/huggingface-roberta-variants/roberta-base/roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


loading weights
Extracting Features from train data
178/178 [==============================] - 34s 177ms/step
Extracting Features from train data
1/1 [==============================] - 0s 122ms/step
loading weights
Extracting Features from train data
178/178 [==============================] - 34s 177ms/step
Extracting Features from train data
1/1 [==============================] - 0s 114ms/step
loading weights
Extracting Features from train data
178/178 [==============================] - 35s 178ms/step
Extracting Features from train data
1/1 [==============================] - 0s 114ms/step
loading weights
Extracting Features from train data
178/178 [==============================] - 34s 178ms/step
Extracting Features from train data
1/1 [==============================] - 0s 109ms/step
loading weights
Extracting Features from train data
178/178 [==============================] - 35s 178ms/step
Extracting Features from train data
1/1 [==============================] - 0s 115ms/step
*****

In [28]:
predictions = model5_predictions * 0.6 + model1_predictions * 0.3 + preds_ragnar * 0.1 # model3_predictions * 0.3 + model2_predictions * 0.25

In [29]:
predictions

array([-0.47250219, -0.43566774, -0.4106286 , -2.45019254, -1.84634897,
       -1.24610696,  0.1523464 ])

In [30]:
np.round(predictions, 4)

array([-0.4725, -0.4357, -0.4106, -2.4502, -1.8463, -1.2461,  0.1523])

In [31]:
submission_df.target = np.round(predictions,4)
print(submission_df)
submission_df.to_csv("submission.csv", index=False)

          id  target
0  c0f722661 -0.4725
1  f0953f0a5 -0.4357
2  0df072751 -0.4106
3  04caf4e0c -2.4502
4  0e63f8bea -1.8463
5  12537fe78 -1.2461
6  965e592c0  0.1523
